# All Experiment Scripts

In [1]:
import experiment_funcs

# SUM:

## Case 1: Balanced 2 tables matching, n=3
#### Matching [Total Max, Total Min] Outcome

In [2]:
experiment_funcs.create_csv_table("trial1sample")
for i in range(5):
    # Create Randomized Data
    table_a_non_duplicated, table_b, table_a_dup = experiment_funcs.create_synth_data(100, 100, "table1", "table2", "table1_dup")
    
    # Bipartite Matching Script
    total_max, total_min, bip_max_matching_time, bip_min_matching_time = experiment_funcs.sum_bip_script(table_a_non_duplicated, table_b, "name", 0.09, 3)

    # Run Naive Matching Script
    naive_total_max, naive_total_min, naive_max_matching_time, naive_min_matching_time = experiment_funcs.sum_naive_script(3, 1, 0.09, "table1_dup", "table2")
    
    # Run Random Matching Script
    sampled_total_max, sampled_total_min, sampled_max_matching_time, sampled_min_matching_time = experiment_funcs.sum_random_sample_script(3, 1, 0.09, 50, "table1_dup", "table2")
    
    # Record Experiment Results
    experiment_funcs.table_csv_output(total_min, total_max, naive_total_min, naive_total_max, sampled_total_min, sampled_total_max, "trial1sample", bip_min_matching_time, bip_max_matching_time, naive_min_matching_time, naive_max_matching_time, sampled_min_matching_time, sampled_max_matching_time)

---- Timing for Graph Construction with Treshold Constraint ----
0.206067 seconds


 'SUM' MAXIMAL MATCHING:
---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----
0.251695 seconds
The number of edges in the graph is: 1608 



 'SUM' MINIMAL MATCHING RESULTS:
True
---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----
0.266438 seconds
The number of edges in the graph is: 1608 

BP Matching: Highest bound for maximum: 1052.0
BP Matching: Lowest bound for minimum: 620.0
Loaded catalogs.
NAIVE MAX MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.172063  seconds
NAIVE MIN MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.134842  seconds
NAIVE MAX Matching Bound:  9690
NAIVE MIN Matching Bound:  4077
Loaded catalogs.
RANDOM SAMPLE MAX MATCHING
Performing random sample match (edit distance)...
Si

# COUNT:

### CASE 1: Balanced 2 tables, matching n=3

In [3]:
experiment_funcs.create_csv_table("count_trialsample")
for i in range(5):
    # Create Randomized Data
    table_a_non_duplicated, table_b, table_a_dup = experiment_funcs.create_synth_data(100, 100, "table1", "table2", "table1_dup")
    
    # Bipartite Matching Script
    total_max, total_min, bip_max_matching_time, bip_min_matching_time = experiment_funcs.count_bip_script(table_a_non_duplicated, table_b, "name", 0.09, 3, 5)

    # Run Naive Matching Script
    naive_total_max, naive_total_min, naive_max_matching_time, naive_min_matching_time = experiment_funcs.count_naive_script(3, 1, 0.09, "table1_dup", "table2", 5)
    
    # Run Random Matching Script
    sampled_total_max, sampled_total_min, sampled_max_matching_time, sampled_min_matching_time = experiment_funcs.count_random_sample_script(3, 1, 0.09, 50, "table1_dup", "table2", 5)
    
    # Record Experiment Results
    experiment_funcs.table_csv_output(total_min, total_max, naive_total_min, naive_total_max, sampled_total_min, sampled_total_max, "count_trialsample", bip_min_matching_time, bip_max_matching_time, naive_min_matching_time, naive_max_matching_time, sampled_min_matching_time, sampled_max_matching_time)

---- Timing for Graph Construction with Treshold Constraint ----
0.227559 seconds
The number of edges in the graph is: 1893 



 'SUM' MAXIMAL MATCHING:
---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----
0.171377 seconds


 'SUM' MINIMAL MATCHING RESULTS:
---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----
0.122302 seconds
BP Matching: Highest bound for maximum: 746.0
BP Matching: Lowest bound for minimum: 118.0 

Loaded catalogs.
NAIVE MAX MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.178765  seconds
NAIVE MIN MATCHING
Performing compare all match (edit distance)...
Naive Edit Distance Matching computation time taken:  0.141509  seconds
NAIVE MAX Matching Bound:  4992
NAIVE MIN Matching Bound:  15165
Loaded catalogs.
RANDOM SAMPLE MAX MATCHING
Performing random sample match (edit distance)...
Simulation-Based Edit Distance Matching computati

## Case 2: 1's table disproportionately bigger than n's table. 
i.e. table_a has 10,000 rows, table_b has 100 rows, n=3, sim_threshold = 0.09

In [4]:
### Data Configuration Here
table_a_non_duplicated, table_b, table_a_dup = experiment_funcs.create_synth_data(10000, 100, "table1", "table2", "table1_dup")
# print(table_a_non_duplicated, table_b, table_a_dup)

In [5]:
#### Bipartite Matching Experiment Scripts
total_max, total_min = experiment_funcs.sum_bip_script(table_a_non_duplicated, table_b, "name", 0.09, 3)
#### Naive Matching Experiment Scripts
naive_total_max, naive_total_min = experiment_funcs.sum_naive_script(3, 1, 0.09, "table1_dup", "table2")
#### Random Sampling Matching Experiment Scripts
sampled_total_max, sampled_total_min = experiment_funcs.sum_random_sample_script(3, 1, 0.09, 50, "table1_dup", "table2")

10.0% complete
20.0% complete
30.0% complete
40.0% complete
50.0% complete
60.0% complete
70.0% complete
80.0% complete
90.0% complete
100.0% complete
110.0% complete
120.0% complete
130.0% complete
140.0% complete
150.0% complete
160.0% complete
170.0% complete
180.0% complete
190.0% complete
200.0% complete
210.0% complete
220.0% complete
230.0% complete
240.0% complete
250.0% complete
260.0% complete
270.0% complete
280.0% complete
---- Timing for Graph Construction with Treshold Constraint ----
20.803282 seconds


 'SUM' MAXIMAL MATCHING:
---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----
10.951397 seconds
The number of edges in the graph is: 135252 



 'SUM' MINIMAL MATCHING RESULTS:
True
---- Timing for Matching (Done on the graph constructed with the treshold constraint) ----
12.44584 seconds
The number of edges in the graph is: 135252 

BP Matching: Highest bound for maximum: 1345.0
BP Matching: Lowest bound for minimum: 477.0


ValueError: too many values to unpack (expected 2)

In [ ]:
"""
Observation: Bipartite Matching gave the lowest outcome. Why?


Matching [Total Max, Total Min] Outcome
"""
print("Bipartite Matching: ", "[", total_min, ", ", total_max, "]")
print("Naive Matching: ", "[", naive_total_min, ", ", naive_total_max, "]")
print("Random Sample Matching: ", "[", sampled_total_min, ", ", sampled_total_max, "]")